# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## План работы

1.	Загрузим и подготовим данные.
2.	Обучим разные модели. Для каждой попробуйем различные гиперпараметры.
3.	Проанализируем скорость работы и качество моделей.

## Описание данных

Признаки:
-	`DateCrawled` — дата скачивания анкеты из базы
-	`VehicleType` — тип автомобильного кузова
-	`RegistrationYear` — год регистрации автомобиля
-	`Gearbox` — тип коробки передач
-	`Power` — мощность (л. с.)
-	`Model` — модель автомобиля
-	`Kilometer` — пробег (км)
-	`RegistrationMonth` — месяц регистрации автомобиля
-	`FuelType` — тип топлива
-	`Brand` — марка автомобиля
-	`Repaired` — была машина в ремонте или нет
-	`DateCreated` — дата создания анкеты
-	`NumberOfPictures` — количество фотографий автомобиля
-	`PostalCode` — почтовый индекс владельца анкеты (пользователя)
-	`LastSeen` — дата последней активности пользователя

Целевой признак:
-    `Price` — цена (евро)

Данные находятся в файле: `/datasets/autos.csv`

## Подготовка данных

In [ ]:
!pip install catboost

In [ ]:
!pip install scikit-learn==1.2.2

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!pip install phik -q

In [ ]:
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor, Pool
from catboost import CatBoostClassifier
from lightgbm import LGBMRegressor
import lightgbm
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
import phik
from phik.report import plot_correlation_matrix
from sklearn.dummy import DummyRegressor
from collections import defaultdict

In [ ]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/autos.csv', delimiter = ',')
pd.set_option('display.max_rows', None)#параметр для выводы всего датафрейма
data.head()

In [ ]:
def inf(data):
    print(f'Размер датасета - {data.shape}')
    print()
    print('Информация о данных:')
    print(data.info())
    print()
    print(f'Количество пропусков в датасете: {data.isna().sum().sum()}')
    print(f'Указание на пропуски в датасете: {data.isna().sum()}')
    print(f'Дубликаты в датасете: {data.duplicated().sum()}')

In [ ]:
inf(data)

In [ ]:
for i in data.select_dtypes(exclude='number').columns.tolist():
    print(f'{i}:')
    print(data[i].unique())
    print()

In [ ]:
data.describe()

# Выводы

- В целевом признаке "Price" и в "Power" минимальное значение - 0;

- В столбце "RegistrationYear" указан странный год регистрации автомобиля, а именно 1000;

- Выявлены 4 дубликата, от которых нужно избавиться;

- В 5-ти стобцах наблюдаются пропуски;

- В столбце "RegistrationMonth" отсутсвует месяц регистрации;

- Названия столбцов удобней и правильней будет привести к нижнему регистру;

- Есть признаки, которые никак не влияют на цену - DateCrawled, RegistrationMonth, DateCreated, NumberOfPictures, PostalCode и LastSeen. От них надо избавиться.


## Предобработка данных

**Приведем названия колонок к нижнему регистру**

In [ ]:
data.columns = data.columns.str.lower()
data.columns

**Как сказано выше, не все признаки влияют на цену(целевой признак), поэтому создадим новый датасет без их учета**

In [ ]:
auto = data[['vehicletype',
            'registrationyear',
            'gearbox',
            'power',
            'model',
            'kilometer',
            'fueltype',
            'brand',
            'repaired',
            'price']]

In [ ]:
def s(auto):
    print(f'Размер датасета - {auto.shape}')
    print()
    print('Информация о данных:')
    print(auto.info())
    print()
    print(f'Количество пропусков в датасете: {auto.isna().sum().sum()}')
    print(f'Указание на пропуски в датасете: {auto.isna().sum()}')
    print(f'Дубликаты в датасете: {auto.duplicated().sum()}')

In [ ]:
s(auto)

**Проведем визуализацию количественных признаков**

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(1, 2, 1)
plt.hist(auto['price'], histtype='bar', bins=50, color='lightsteelblue', edgecolor='black')
plt.xlabel('Цена, евро')
plt.ylabel('Количество автомобилей')
plt.title('Распределение рыночной стоимости автомобиля в евро')

plt.subplot(1, 2, 2)
plt.boxplot(auto['price'])

plt.title('"Ящик с усами" ')

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(1,2,1)
plt.hist(auto['registrationyear'],histtype='bar',bins=40,range = (1970,2022),color = 'hotpink',edgecolor = "black")
plt.xlabel('Год выпуска')
plt.ylabel('Количество автомобилей')
plt.title('Распределение возраста автомобиля в годах')

plt.subplot(1,2,2)
plt.boxplot(auto['registrationyear'])

plt.title('"Ящик с усами" ')

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(1,2,1)
plt.hist(auto['power'],histtype='bar',range = (0,400),bins=40,color = 'mediumorchid',edgecolor = "black")
plt.xlabel('Мощность,л.с')
plt.ylabel('Количество автомобилей')
plt.title('Распределение автомобилей по мощностям')

plt.subplot(1,2,2)
plt.boxplot(auto['power'])
plt.title('"Ящик с усами" ')

plt.show()

In [ ]:
plt.figure(figsize=(10, 8))

plt.subplot(1,2,1)
plt.hist(auto['kilometer'],histtype='bar',bins=40,color = 'powderblue',edgecolor = "black")
plt.xlabel('Пробег,км')
plt.ylabel('Количество автомобилей')
plt.title('Распределение автомобилей по пробегу')

plt.subplot(1,2,2)
plt.boxplot(auto['kilometer'])
plt.title('"Ящик с усами" ')

plt.show()

In [ ]:
auto['price'].describe()

In [ ]:
auto['price'].value_counts()

In [ ]:
auto = auto.query('price > 0')

In [ ]:
auto.price.hist(bins=200, figsize=(20,5))
plt.figure(figsize=(10,3))
sns.boxplot(data=auto.price, orient='h')
plt.title('Ящик с усами для целевого признака')
plt.show()

In [ ]:
auto['price'].quantile([0.05, 0.95])

In [ ]:
auto = auto.query('350 <= price <= 14800')
auto['price'].describe()

In [ ]:
auto['registrationyear'].value_counts()

In [ ]:
auto['registrationyear'].describe()

In [ ]:
auto['registrationyear'].quantile([0.05, 0.95])

In [ ]:
auto = auto.query('1993 <= registrationyear <= 2016')
auto['registrationyear'].describe()

In [ ]:
auto['power'].describe()

In [ ]:
auto['power'].quantile([0.05, 0.95])

**По имеющейся информации в интернете об автомобильных двигателях, исключим автомобили с мощностью менее 20 лошадиных сил**

In [ ]:
auto = auto.query('20 <= power <= 209')
auto['power'].describe()

In [ ]:
auto['gearbox'].describe()

**Заменим пропуки на 'manual'**

In [ ]:
auto.fillna({'gearbox':'manual'},inplace=True)

In [ ]:
auto['gearbox'].isna().sum()

In [ ]:
auto['repaired'].describe()

**Заменин пропуски на 'no'**

In [ ]:
auto.fillna({'repaired':'no'},inplace=True)

**Заменин пропуски на 'unknown'**

In [ ]:
auto['fueltype'].describe()

**Заменим пропуски на 'other'**

In [ ]:
auto.fillna({'fueltype':'other'},inplace=True)

In [ ]:
print(*auto['vehicletype'].unique(), sep='\n')

In [ ]:
auto.query('vehicletype != vehicletype')

**Обноружили пропуски в данных типа кузова автомобилей. Провели их заполнение по самому распространенному значению каждого бренда**

In [ ]:
s = auto.query('brand == brand')
sp = s['brand'].unique()
for element in sp:
    df = auto.loc[auto['brand'] == element]
    print(element,' - ',df['vehicletype'].value_counts().idxmax())
    auto.loc[(auto['brand'] == element) & (auto['vehicletype'] != auto['vehicletype']), 'vehicletype'] = df['vehicletype'].value_counts().idxmax()

In [ ]:
auto['vehicletype'].isnull().sum()

In [ ]:
display(auto.loc[auto['brand'] == 'sonstige_autos'])

In [ ]:
df = auto.loc[auto['brand'] == 'sonstige_autos']
print(*df['model'].unique(), sep='\n')

**В данных бренда под обозначением другой авто отсутствует информация о модели автомобиля.
Заполним**

In [ ]:
auto.loc[auto['brand'] == 'sonstige_autos', 'model'] = 'not'

In [ ]:
auto['brand'].value_counts()

In [ ]:
som = auto.query('brand == brand')
tom = som['brand'].unique()
for element in tom:
    df = auto.loc[auto['brand'] == element]
    df.dropna(subset=['model'], inplace=True)
    print(element,' - ',df['model'].value_counts().idxmax())
    auto.loc[((auto['brand'] == element) & (auto['model'] != auto['model'])), 'model'] = df['model'].value_counts().idxmax()

In [ ]:
auto['model'].isnull().sum()

In [ ]:
auto.info()

In [ ]:
auto.isnull().sum()

**Все пропуски обработаны, можно спокойно переходить к обучению моделей!**

In [ ]:
file = auto
all_features = list(file.columns)
correlation = file[all_features].phik_matrix(interval_cols = [ 'power', 'price'])
plt.figure(figsize=(12,10))
colormap = sns.color_palette("Set2_r")
plt.rc('font', size= 12)
chart = sns.heatmap(correlation, annot=True, cmap=colormap, linewidths=1, linecolor='black')
plt.title(r'Корреляция между признаками $\phi_K$', fontsize=15, fontweight="bold", color='pink')
chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
chart.set_yticklabels(chart.get_xticklabels(), rotation=0, horizontalalignment='right')
plt.show()

**Корреляционный анализ показывает слабую связь между признаком, указыающим была машина в ремонте или нет и почти всеми остальными признаками из датасета,  а высокую между моделью автомобиля типом автомобильного кузова, еще между моделью автомобиля и мощностью.**

**Вывод по предобработке данных:**

- Избавилась от признаков,не влияющих на целевой, дубликатов, выбрасов;
- Привела столбцы к нижнему регистру;
- Заполнила пропуски в gearbox,repaired,model, vehicletype, fueltype;
- Корреляционный анализ показывает слабую связь между целевым признаком и пробегом(kilometer),  а высокую корреляцию с возрастом автомобиля (registrationyear)

## Обучение моделей

In [ ]:
data_ohe = pd.get_dummies(auto, drop_first=True)
target = data_ohe['price']
features = data_ohe.drop('price', axis=1)

In [ ]:
data_train, data_valid = train_test_split(data_ohe, test_size=0.2, random_state=12345)
data_valid, data_test = train_test_split(data_valid , test_size=0.2, random_state=12345)

features_train = data_train.drop(columns=['price'])
target_train = data_train['price']

features_valid = data_valid.drop(columns=['price'])
target_valid = data_valid['price']

features_test = data_test.drop(columns=['price'])
target_test = data_test['price']

## Линейная регрессия

In [ ]:
def startik(model, features_train, target_train, features_valid, target_valid):
    
    start = time.time()    
    model.fit(features_train, target_train)
    training_time = time.time() - start
    
    start = time.time()  
    target_pred = model.predict(features_valid)
    predict_time = time.time() - start

    return training_time, predict_time


In [ ]:
model = LinearRegression()
start_time = time.time()
model.fit(features_train, target_train)
training_time_lr, predict_time_lr = startik(model, features_train, target_train, features_valid, target_valid)
predict_lr = pd.DataFrame(model.predict(features_valid))
score_lr = mean_squared_error(target_valid, predict_lr) ** 0.5
print("Время обучения:", training_time_lr)
print("Время предсказания:", predict_time_lr)
print("RMSE:", score_lr)

## LightGBM

In [ ]:
d = defaultdict(list)
lgbm_model = LGBMRegressor(num_leaves=31, learning_rate=0.05, n_estimators=20, random_state=12345)
start_time = time.time()
lgbm_model.fit(features_train, target_train)
training_time_lgbm, predict_time_lgbm = startik(lgbm_model, features_train, target_train, features_valid, target_valid)
predict_lgbm = pd.DataFrame(lgbm_model.predict(features_valid))
score_lgbm = mean_squared_error(target_valid, predict_lgbm) ** 0.5
print("Время обучения:", training_time_lgbm)
print("Время предсказания:", predict_time_lgbm)
print("RMSE:", score_lgbm)

**Чтобы оптимизировать несколько гиперпараметров модели, можно использовать вложенный цикл с использованием valid выборки. Внутренний цикл будет перебирать значения одного гиперпараметра, а внешний цикл будет перебирать значения другого гиперпараметра**
**Используем для CatBoost**


## CatBoost

In [ ]:
best_score = 2500
for learning_rate in [0.01, 0.05, 0.1]:
    for iterations in [50, 100, 200]:
        cat_model = CatBoostRegressor(loss_function="RMSE", iterations=iterations, learning_rate=learning_rate, random_state=12345)
        start_time = time.time()
        cat_model.fit(features_train, target_train, verbose=10)
        training_time, predict_time = startik(cat_model, features_train, target_train, features_valid, target_valid)
        predict_cat = cat_model.predict(features_valid)
        score_cat = mean_squared_error(target_valid, predict_cat) ** 0.5
        
        if score_cat < best_score:
            best_score = score_cat
            best_learning = learning_rate
            best_iterations = iterations
            training_time_cat = training_time
            predict_time_cat = predict_time

print("learning rate:", best_learning)
print("iterations:", best_iterations)
print("Время обучения:", training_time_cat)
print("Время предсказания:", predict_time_cat)
print("RMSE:", best_score)

## Анализ моделей

In [ ]:
index = ['LinearRegression', 'CatBoostRegressor', 'LGBMRegressor']

data = {'RMSE':[score_lgbm, best_score, score_lgbm],

        'Время обучения ':[training_time_lr, training_time_cat, training_time_lgbm],
        
        'Время предсказания модели':[predict_time_lr,predict_time_cat,predict_time_lgbm]

       }
som_data = pd.DataFrame(data=data, index=index)
display(som_data)

**У модели CatBoost лучшие время предсказания модели и RMSE, но по времени обучения остает от других, что не сказать про LinearRegression, у которой оно быстрее всех. У Linear почти одинаковый резульат RMSE с LGBM, но модель предсказывает дольше всех.
Исходя из условий задачи, заказчику важны:**
- 1. качество предсказания;
- 2. скорость предсказания;
- 3. время обучения.

**Следовательно,порядок критериев также учитывает их приоритет, значит отдадим предпочтение CatBoostRegressor.**

**Проверим CatBoostRegressor на тестовой выборке**

In [ ]:
features_test = data_ohe.drop('price', axis=1)
target_test = data_ohe['price']

In [ ]:
cat_model = CatBoostRegressor(loss_function="RMSE",learning_rate=0.1, iterations=200, random_state=12345)
start_time = time.time()
cat_model.fit(features_train, target_train, verbose=10)
training_time_cat_t, predict_time_cat_t = startik(cat_model, features_train, target_train, features_test, target_test)
predict_cat_t = cat_model.predict(features_test)
score_cat_t = mean_squared_error(target_test, predict_cat_t) ** 0.5
print("Время обучения:", training_time_cat_t)
print("Время предсказания:", predict_time_cat_t)
print("RMSE:", score_cat_t)

In [ ]:
index = ['LinearRegression', 'CatBoostRegressor', 'LGBMRegressor','Лучшая модель на тестовой выборке']

data = {'RMSE':[score_lgbm, best_score, score_lgbm, score_cat_t],

        'Время обучения ':[training_time_lr, training_time_cat, training_time_lgbm,training_time_cat_t],
        
        'Время предсказания модели':[predict_time_lr,predict_time_cat,predict_time_lgbm,predict_time_cat_t]

       }
som_data = pd.DataFrame(data=data, index=index)
display(som_data)

## Создадим модель  генерирования константных предсказаний.

In [ ]:
dummy = DummyRegressor(strategy='mean')
start_time = time.time()
dummy.fit(features_train, target_train)
training_time_d = time.time() - start_time
start_time = time.time()
predict_d = dummy.predict(features_test)
predict_time_d = time.time() - start_time
score_d = mean_squared_error(target_test, predict_d) ** 0.5
print("Время обучения:", training_time_d)
print("Время предсказания:", predict_time_d)
print("RMSE:", score_d)

In [ ]:
index = ['Лучшая модель на тестовой выборке','Mодель генерирования константных предсказаний']

data = {'RMSE':[score_cat_t,score_d],

        'Время обучения ':[training_time_cat_t,training_time_d],
        
        'Время предсказания':[predict_time_cat,predict_time_d]

       }
som_data = pd.DataFrame(data=data, index=index)
display(som_data)

**Заметим, что результат тестирования нашей модели на тествой выборке лучше, чем результат константной модели, так как RMSE=3452, а должен быть по ТЗ меньше 2500!**

## Итоговый вывод

**В данном проекты были проведены следующие действия:**

  - Избавились от признаков,не влияющих на целевой, дубликатов, выбрасов;
  - Приведены столбцы к нижнему регистру;
  - Заполнены пропуски в gearbox,repaired,model, vehicletype, fueltype;
  - Корреляционный анализ показал слабую связь между целевым признаком и пробегом(kilometer),  а высокую корреляцию с возрастом автомобиля (registrationyear);
  - У модели CatBoost лучшие время предсказания модели и RMSE, но по времени обучения остает от других, что не сказать про LinearRegression, у которой оно быстрее всех. У Linear почти одинаковый резульат RMSE с LGBM, но модель предсказывает дольше всех.;
  - Исходя из условий задачи, заказчику важны: качество предсказания,скорость предсказания,время обучения,cледовательно,порядок критериев также учитывает их приоритет, значит лучшая модель CatBoostRegressor;
  - Значение метрики RMSE везде меньше 2500;
  - Проверили CatBoostRegressor на тестовой выборке;
  - На тренировочной выборке RMSE CatBoost = 1274.074160,  а на тестовой = 1284.759719, а результат константной модели показал = 3452.677196 	.